# Quick FIFA Insights & Visualizations
## Interactive charts ready for Hex platform

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Load data
results_df = pd.read_csv('../data/raw/results.csv', parse_dates=['date'])
goalscorers_df = pd.read_csv('../data/raw/goalscorers.csv', parse_dates=['date'])
shootouts_df = pd.read_csv('../data/raw/shootouts.csv', parse_dates=['date'])

results_df['year'] = results_df['date'].dt.year
results_df['total_goals'] = results_df['home_score'] + results_df['away_score']

print(f"✓ Loaded {len(results_df):,} matches, {len(goalscorers_df):,} goals")
print(f"Date range: {results_df['date'].min()} to {results_df['date'].max()}")

## Top Teams Performance (Last 4 Years)

In [ ]:
# Recent performance (2022-2025)
recent_df = results_df[results_df['year'] >= 2022].copy()

# Calculate win rates for all teams
team_stats = []

for team in set(recent_df['home_team'].unique()) | set(recent_df['away_team'].unique()):
    home_matches = recent_df[recent_df['home_team'] == team]
    away_matches = recent_df[recent_df['away_team'] == team]
    
    home_wins = (home_matches['home_score'] > home_matches['away_score']).sum()
    away_wins = (away_matches['away_score'] > away_matches['home_score']).sum()
    total_matches = len(home_matches) + len(away_matches)
    
    if total_matches >= 10:  # Filter teams with at least 10 matches
        goals_scored = home_matches['home_score'].sum() + away_matches['away_score'].sum()
        goals_conceded = home_matches['away_score'].sum() + away_matches['home_score'].sum()
        
        team_stats.append({
            'Team': team,
            'Matches': total_matches,
            'Wins': home_wins + away_wins,
            'Win Rate': (home_wins + away_wins) / total_matches * 100,
            'Goals Scored': goals_scored,
            'Goals/Match': goals_scored / total_matches,
            'Goal Difference': goals_scored - goals_conceded
        })

team_df = pd.DataFrame(team_stats).sort_values('Win Rate', ascending=False).head(20)

# Interactive bar chart
fig = px.bar(team_df, 
             x='Win Rate', 
             y='Team',
             orientation='h',
             color='Win Rate',
             color_continuous_scale='RdYlGn',
             title='Top 20 Teams by Win Rate (2022-2025)',
             hover_data=['Matches', 'Wins', 'Goals/Match'])

fig.update_layout(height=600, yaxis={'categoryorder':'total ascending'})
fig.show()

display(team_df[['Team', 'Matches', 'Win Rate', 'Goals/Match', 'Goal Difference']].head(15))

## World Cup Evolution: Goals & Trends

In [ ]:
# World Cup specific analysis
wc_df = results_df[results_df['tournament'] == 'FIFA World Cup'].copy()
wc_yearly = wc_df.groupby('year').agg({
    'total_goals': ['mean', 'sum'],
    'date': 'count'
}).round(2)

wc_yearly.columns = ['Avg Goals/Match', 'Total Goals', 'Matches']
wc_yearly = wc_yearly.reset_index()

# Dual-axis chart
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=wc_yearly['year'], y=wc_yearly['Avg Goals/Match'],
               mode='lines+markers', name='Avg Goals/Match',
               line=dict(color='#FF6B6B', width=3),
               marker=dict(size=10)),
    secondary_y=False
)

fig.add_trace(
    go.Bar(x=wc_yearly['year'], y=wc_yearly['Matches'],
           name='Total Matches', marker_color='#4ECDC4', opacity=0.6),
    secondary_y=True
)

fig.update_layout(
    title='FIFA World Cup: Scoring Trends Over Time',
    hovermode='x unified',
    height=500
)

fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='Average Goals per Match', secondary_y=False)
fig.update_yaxes(title_text='Total Matches', secondary_y=True)

fig.show()

print(f"\n📊 Key Stats:")
print(f"Highest scoring World Cup: {wc_yearly.loc[wc_yearly['Avg Goals/Match'].idxmax(), 'year']:.0f} ({wc_yearly['Avg Goals/Match'].max():.2f} goals/match)")
print(f"Most matches in one WC: {wc_yearly['Matches'].max():.0f} matches")
print(f"2026 Format: 48 teams → Estimated 104 matches (vs current 64)")

## Top Goal Scorers Analysis

In [ ]:
# Top scorers all-time
top_scorers = goalscorers_df.groupby('scorer').size().sort_values(ascending=False).head(20)

fig = go.Figure(go.Bar(
    x=top_scorers.values,
    y=top_scorers.index,
    orientation='h',
    marker=dict(color=top_scorers.values, colorscale='Viridis')
))

fig.update_layout(
    title='Top 20 International Goal Scorers of All Time',
    xaxis_title='Goals',
    yaxis_title='Player',
    yaxis={'categoryorder':'total ascending'},
    height=600
)

fig.show()

# Penalty analysis
penalty_stats = goalscorers_df.groupby('penalty').size()
print(f"\n⚽ Penalty Stats:")
print(f"Total goals: {len(goalscorers_df):,}")
print(f"Penalty goals: {penalty_stats.get(True, 0):,} ({penalty_stats.get(True, 0)/len(goalscorers_df)*100:.1f}%)")
print(f"Open play goals: {penalty_stats.get(False, 0):,} ({penalty_stats.get(False, 0)/len(goalscorers_df)*100:.1f}%)")

## 2026 World Cup: Key Predictions & Insights

Based on historical data and recent performance (2022-2025)

In [ ]:
# Create predictions dashboard
top_contenders = team_df.head(10).copy()

fig = go.Figure()

# Win rate
fig.add_trace(go.Bar(
    name='Win Rate %',
    x=top_contenders['Team'],
    y=top_contenders['Win Rate'],
    marker_color='#2ECC71'
))

# Goals per match (scaled to percentage for comparison)
fig.add_trace(go.Scatter(
    name='Goals/Match (x20)',
    x=top_contenders['Team'],
    y=top_contenders['Goals/Match'] * 20,
    mode='lines+markers',
    marker=dict(size=12, color='#E74C3C'),
    line=dict(width=3)
))

fig.update_layout(
    title='2026 World Cup Top Contenders<br><sub>Based on 2022-2025 Performance</sub>',
    xaxis_title='Team',
    yaxis_title='Percentage / Scaled Metric',
    barmode='group',
    height=500,
    hovermode='x unified'
)

fig.show()

print("\n🏆 2026 World Cup Predictions:")
print(f"\nTop 5 Favorites:")
for idx, row in top_contenders.head(5).iterrows():
    print(f"  {row['Team']}: {row['Win Rate']:.1f}% win rate, {row['Goals/Match']:.2f} goals/match")

print(f"\n💡 Key Insights:")
print(f"• 48-team format means more upsets and surprises")
print(f"• Traditional powerhouses face expanded competition")
print(f"• Recent form matters: Teams with 70%+ win rate are strong contenders")
print(f"• Goal-scoring efficiency crucial in knockout stages")